In [1278]:
import sys
sys.path.append("../")
import nanonis
import numpy as np
import scipy as sc
import pandas as pd
import matplotlib.pyplot as plt
import xlrd
import lmfit
from lmfit import Model
import glob, os
import matplotlib.colors as colors
from scipy import constants as const
from matplotlib.widgets import Button, Slider
%matplotlib qt
%reload_ext autoreload
%autoreload 2
spectra=nanonis.biasSpectroscopy()
path1="//sannet01.nanogune.intranet/CIC09/STMDATA/Berlino/Pb(111)/2023-11-20 - Pb on graphene/2023-11-30 - Pb on Gr-SiC0001 - sandra2"
path2="//sannet01.nanogune.intranet/CIC09/STMDATA/Berlino/Pb(111)/2023-11-20 - Pb on graphene/2023-12-07 - Pb on Gr-SiC0001 - sandra2"
path3="//sannet01.nanogune.intranet/CIC09/STMDATA/Berlino/Pb(111)/2023-11-20 - Pb on graphene/2023-12-19 - Pb on Gr-SiC0001 - sandra2"

# Theory program

In [1279]:
def find_nearest(array,value):
        idx = (np.abs(array-value)).argmin()
        return idx

def fast_spec(file):
        spectra.load(file)
        x=np.array(spectra.bias)
        y=np.array(spectra.conductance)
        return x,y

def fast_cur(file):
        spectra.load(file)
        x=np.array(spectra.bias)
        y=np.array(spectra.current)
        return x,y

def fast_I(file):
        spectra.load(file)
        x=np.array(spectra.bias)
        y=np.array(spectra.biasVI_b)
        z=np.array(spectra.biasVI_f)
        w=np.array(spectra.current)
        a=np.abs((w[0]-w[-1])/(x[0]-x[-1]))*1e6
        x=x*a
        return x,y,z,w,a

color_list=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

### Badaezpada

In [ ]:
def bcs(delta,x,eta):
    if delta<=0.01: 
        return 1
    else:
        return (np.sign(x))*np.imag(np.divide(np.abs(x+eta*1j),np.sqrt(delta**2-(x+eta*1j)**2)))
    
def bcs_i(delta,x,eta,n):
    if int(np.mod(np.abs(np.around(n)),2))==0:
        if delta<=0.01: 
            return 1
        else:
            return (np.sign(x))*np.imag(np.divide(np.abs(x+eta*1j),np.sqrt(delta**2-(x+eta*1j)**2)))
    else:
        return 1

def fermi(T,x):
    if T==0.0:
        return np.heaviside(-x,1)
    else:
        return np.divide(1,1+np.exp(x/T))

def E1(C1,C2,V,n,Q0):
    k=C2/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def E2(C1,C2,V,n,Q0):
    k=C1/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def Gamma1(V,R1,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta):
    x=np.linspace(-4*np.max(V),4*np.max(V),50000)
    a,b=np.meshgrid(x,E1(C1,C2,V,n,Q0,Delta))
    t=a+b
    return np.dot( bcs_i(Delta,t,eta,n+Q0)*fermi(T,-t),fermi(T,x) )/R1

def Gamma2(V,R2,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta):
    x=np.linspace(-4*np.max(V),4*np.max(V),50000)
    a,b=np.meshgrid(x,E2(C1,C2,V,-n,-Q0,Delta))
    t=a+b
    return np.dot((fermi(T,-t)),bcs_i(Delta,x,eta,n+Q0)*fermi(T,x))/R2



def p(V,R1,R2,C1,C2,Q0,Delta,T):
    m1=(Gamma1(V,R1,C1,C2,0,Q0,Delta,T)+Gamma2(-V,R2,C1,C2,0,-Q0,Delta,T))/(Gamma1(-V,R1,C1,C2,1,-Q0,Delta,T)+Gamma2(V,R2,C1,C2,-1,Q0,Delta,T))
    e1=(Gamma1(-V,R1,C1,C2,0,-Q0,Delta,T)+Gamma2(V,R2,C1,C2,0,Q0,Delta,T))/(Gamma1(V,R1,C1,C2,1,Q0,Delta,T)+Gamma2(-V,R2,C1,C2,-1,-Q0,Delta,T))
    e2=(Gamma1(-V,R1,C1,C2,-1,-Q0,Delta,T)+Gamma2(V,R2,C1,C2,1,Q0,Delta,T))/(Gamma1(V,R1,C1,C2,2,Q0,Delta,T)+Gamma2(-V,R2,C1,C2,-2,-Q0,Delta,T))
    m2=(Gamma1(V,R1,C1,C2,-1,Q0,Delta,T)+Gamma2(-V,R2,C1,C2,1,-Q0,Delta,T))/(Gamma1(-V,R1,C1,C2,2,-Q0,Delta,T)+Gamma2(V,R2,C1,C2,-2,Q0,Delta,T))
    e3=(Gamma1(-V,R1,C1,C2,-2,-Q0,Delta,T)+Gamma2(V,R2,C1,C2,2,Q0,Delta,T))/(Gamma1(V,R1,C1,C2,3,Q0,Delta,T)+Gamma2(-V,R2,C1,C2,-3,-Q0,Delta,T))
    m3=(Gamma1(V,R1,C1,C2,-2,Q0,Delta,T)+Gamma2(-V,R2,C1,C2,2,-Q0,Delta,T))/(Gamma1(-V,R1,C1,C2,3,-Q0,Delta,T)+Gamma2(V,R2,C1,C2,-3,Q0,Delta,T))
    e4=(Gamma1(-V,R1,C1,C2,-3,-Q0,Delta,T)+Gamma2(V,R2,C1,C2,3,Q0,Delta,T))/(Gamma1(V,R1,C1,C2,4,Q0,Delta,T)+Gamma2(-V,R2,C1,C2,-4,-Q0,Delta,T))
    m4=(Gamma1(V,R1,C1,C2,-3,Q0,Delta,T)+Gamma2(-V,R2,C1,C2,3,-Q0,Delta,T))/(Gamma1(-V,R1,C1,C2,4,-Q0,Delta,T)+Gamma2(V,R2,C1,C2,-4,Q0,Delta,T))
    p0=np.divide(1,1+m1+e1+m1*m2+e1*e2+m1*m2*m3+e1*e2*e3+m1*m2*m3*m4+e1*e2*e3*e4)
    p1=np.multiply(p0,e1)
    p_1=np.multiply(p0,m1)
    p2=np.multiply(p0,e1*e2)
    p_2=np.multiply(p0,m1*m2)
    p3=np.multiply(p0,e1*e2*e3)
    p_3=np.multiply(p0,m1*m2*m3)
    p4=np.multiply(p0,e1*e2*e3*e4)
    p_4=np.multiply(p0,m1*m2*m3*m4)
    return p0,p_1,p1,p_2,p2,p_3,p3,p_4,p4

def current(V,R1,R2,C1,C2,Q0,Delta,T):
    p0,p_1,p1,p_2,p2,p_3,p3,p_4,p4=p(V,R1,R2,C1,C2,Q0,Delta,T)
    G0=Gamma1(V,R1,C1,C2,0,Q0,Delta,T)-Gamma1(-V,R1,C1,C2,0,-Q0,Delta,T)
    G1=Gamma1(V,R1,C1,C2,1,Q0,Delta,T)-Gamma1(-V,R1,C1,C2,-1,-Q0,Delta,T)
    G_1=Gamma1(V,R1,C1,C2,-1,Q0,Delta,T)-Gamma1(-V,R1,C1,C2,1,-Q0,Delta,T)
    G2=Gamma1(V,R1,C1,C2,2,Q0,Delta,T)-Gamma1(-V,R1,C1,C2,-2,-Q0,Delta,T)
    G_2=Gamma1(V,R1,C1,C2,-2,Q0,Delta,T)-Gamma1(-V,R1,C1,C2,2,-Q0,Delta,T)
    G3=Gamma1(V,R1,C1,C2,3,Q0,Delta,T)-Gamma1(-V,R1,C1,C2,-3,-Q0,Delta,T)
    G_3=Gamma1(V,R1,C1,C2,-3,Q0,Delta,T)-Gamma1(-V,R1,C1,C2,3,-Q0,Delta,T)
    G4=Gamma1(V,R1,C1,C2,4,Q0,Delta,T)-Gamma1(-V,R1,C1,C2,-4,-Q0,Delta,T)
    G_4=Gamma1(V,R1,C1,C2,-4,Q0,Delta,T)-Gamma1(-V,R1,C1,C2,4,-Q0,Delta,T)
    return np.multiply(p0,G0)+np.multiply(p1,G1)+np.multiply(p_1,G_1)+np.multiply(p2,G2)+np.multiply(p_2,G_2)+np.multiply(p3,G3)+np.multiply(p_3,G_3)+np.multiply(p4,G4)+np.multiply(p_4,G_4)

def current2(V,R1,R2,C1,C2,Q0,Delta,T):
    p0,p_1,p1,p_2,p2,p_3,p3,p_4,p4=p(V,R1,R2,C1,C2,Q0,Delta,T)
    G0=Gamma2(V,R2,C1,C2,0,Q0,Delta,T)-Gamma2(-V,R2,C1,C2,0,-Q0,Delta,T)
    G1=Gamma2(V,R2,C1,C2,1,Q0,Delta,T)-Gamma2(-V,R2,C1,C2,-1,-Q0,Delta,T)
    G_1=Gamma2(V,R2,C1,C2,-1,Q0,Delta,T)-Gamma2(-V,R2,C1,C2,1,-Q0,Delta,T)
    G2=Gamma2(V,R2,C1,C2,2,Q0,Delta,T)-Gamma2(-V,R2,C1,C2,-2,-Q0,Delta,T)
    G_2=Gamma2(V,R2,C1,C2,-2,Q0,Delta,T)-Gamma2(-V,R2,C1,C2,2,-Q0,Delta,T)
    G3=Gamma2(V,R2,C1,C2,3,Q0,Delta,T)-Gamma2(-V,R2,C1,C2,-3,-Q0,Delta,T)
    G_3=Gamma2(V,R2,C1,C2,-3,Q0,Delta,T)-Gamma2(-V,R2,C1,C2,3,-Q0,Delta,T)
    G4=Gamma2(V,R2,C1,C2,4,Q0,Delta,T)-Gamma2(-V,R2,C1,C2,-4,-Q0,Delta,T)
    G_4=Gamma2(V,R2,C1,C2,-4,Q0,Delta,T)-Gamma2(-V,R2,C1,C2,4,-Q0,Delta,T)
    return np.multiply(p0,G0)+np.multiply(p1,G1)+np.multiply(p_1,G_1)+np.multiply(p2,G2)+np.multiply(p_2,G_2)+np.multiply(p3,G3)+np.multiply(p_3,G_3)+np.multiply(p4,G4)+np.multiply(p_4,G_4)


### Full theory program

In [1373]:
def bcs(delta,x,eta):
    if delta<=0.01: 
        return 1
    else:
        return (np.sign(x))*np.imag(np.divide(np.abs(x+eta*1j),np.sqrt(delta**2-(x+eta*1j)**2)))
    
def bcs_i(delta,x,eta,n):
    if np.mod(n,2)==0:
        if delta<=0.01: 
            return 1
        else:
            return (np.sign(x))*np.imag(np.divide(np.abs(x+eta*1j),np.sqrt(delta**2-(x+eta*1j)**2)))
    else:
        return 1
        

def fermi(T,x):
    if T==0.0:
        return np.heaviside(-x,1)
    else:
        return np.divide(1,1+np.exp(x/T))

def E1(C1,C2,V,n,Q0,Delta):
    k=C2/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)#+(1-(-1)**n)*Delta/2

def E2(C1,C2,V,n,Q0,Delta):
    k=C1/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)#+(1-(-1)**n)*Delta/2

def Gamma1D(V,R1,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta):
    x=np.linspace(-4*np.max(V),4*np.max(V),10000)
    a,b=np.meshgrid(x,E1(C1,C2,V,n,Q0,Delta))
    t=a+b
    return np.dot( bcs_i(Delta,t,eta,n)*(fermi(T,-t)),bcs(delta_t,x,eta)*fermi(T,x) )/R1

def Gamma2D(V,R2,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta):
    x=np.linspace(-4*np.max(V),4*np.max(V),10000)
    a,b=np.meshgrid(x,E2(C1,C2,V,-n,-Q0,Delta))
    t=a+b
    return np.dot(bcs(delta_s,t,eta)*(fermi(T,-t)),bcs_i(Delta,x,eta,n)*fermi(T,x))/R2


def PND(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n):
    mn=(Gamma1D(V,R1,C1,C2,-n,Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(-V,R2,C1,C2,n,-Q0,Delta,delta_t,delta_s,T,eta))/(Gamma1D(-V,R1,C1,C2,n+1,-Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(V,R2,C1,C2,-n-1,Q0,Delta,delta_t,delta_s,T,eta))
    en=(Gamma1D(-V,R1,C1,C2,-n,-Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(V,R2,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta))/(Gamma1D(V,R1,C1,C2,n+1,Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(-V,R2,C1,C2,-n-1,-Q0,Delta,delta_t,delta_s,T,eta))
    return en,mn

def check_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta):
    Vmax=V[-1]
    n=[0]
    while True:
        a=[]
        b=[]
        for i in n:
            an,bn=PND(Vmax,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,i)
            a.append(an[0])
            b.append(bn[0])
        p0=1
        for i in range(len(a)):
            temp=np.zeros(len(a))
            temp[0:i+1]=1
            temp=temp.tolist()
            p0+=np.prod(a,where=temp)+np.prod(b,where=temp)
        p0=1/p0
        pn=p0*np.prod(a)
        p_n=p0*np.prod(b)
        #print(pn,p_n)
        n.append(n[-1]+1)
        if pn<0.0001 and p_n<0.0001:
            #print(len(a))
            break 
    return n[-1]

def all_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n):
    a=[]
    b=[]
    for i in range(n):
        an,bn=PND(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,i)
        a.append(an)
        b.append(bn)
    p0=1
    pn=[]
    p_n=[]
    for i in range(n):
        temp1=np.full(len(V),1.0)
        temp2=np.full(len(V),1.0)
        for j in range(i+1):
            temp1*=a[j]     
            temp2*=b[j]
        p0+=temp1+temp2
        pn.append(temp1)
        p_n.append(temp2)
    p0=1/p0
    pn=p0*np.array(pn)
    p_n=p0*np.array(p_n)
    return p0,pn,p_n

def G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n):
    return Gamma1D(V,R1,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta)-Gamma1D(-V,R1,C1,C2,-n,-Q0,Delta,delta_t,delta_s,T,eta)

def G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n):
    return Gamma2D(V,R2,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta)-Gamma2D(-V,R2,C1,C2,-n,-Q0,Delta,delta_t,delta_s,T,eta)

def currentD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta):
    n=check_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta)
    p0,pn,p_n=all_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n)
    I=p0*G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,i)
        I+=p_n[i-1]*G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,-i)
    t=np.gradient(I)
    return t/np.sum(t)


def currentD2(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta):
    n=check_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta)
    p0,pn,p_n=all_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n)
    I=p0*G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,i)
        I+=p_n[i-1]*G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,-i)
    t=np.gradient(I)
    return t/np.sum(t)

In [1374]:
V=np.linspace(-5,5,50)
R1=1000
R2=1
C1=0.01
C2=0.8
Q0=0.44
Delta=0.8
delta_t=1.3
delta_s=0.3
T=0.15
eta=0.15

xx,yy=fast_spec(ifar[70])
plt.plot(1000*xx,yy/yy[-1])

y=currentD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta)
plt.plot(V,y/y[0])



## Simple understanding progam

In [1293]:
def par(n,q0,Ec,delta):
    return Ec*(n-q0)**2+delta*(1-(-1)**n)/2
def find_oc(q0,Ec,delta):
    a=int(q0)
    e1=par(a-1,q0,Ec,delta)
    e2=par(a,q0,Ec,delta)
    e3=par(a+1,q0,Ec,delta)
    b=[e1,e2,e3]
    return a+b.index(min(b))-1

def oc(q0,Ec,delta,T):
    a=round(q0)
    e1=par(a-1,q0,Ec,delta)
    e2=par(a,q0,Ec,delta)
    e3=par(a+1,q0,Ec,delta)
    b=np.array([e1,e2,e3])
    Z=sum(np.exp(-b/T))
    return np.exp(-b/T)/Z,a


def exc(q0,Ec,delta,T,deltat):
    b,a=oc(q0,Ec,delta,T)
    e0=par(a-2,q0,Ec,delta)
    e1=par(a-1,q0,Ec,delta)
    e2=par(a,q0,Ec,delta)
    e3=par(a+1,q0,Ec,delta)
    e4=par(a+2,q0,Ec,delta)
    return [b[0],b[0],b[1],b[1],b[2],b[2]],[np.abs(e2-e1)+deltat,-np.abs(e1-e0)-deltat,np.abs(e3-e2)+deltat,-np.abs(e1-e2)-deltat,np.abs(e4-e3)+deltat,-np.abs(e2-e3)-deltat],[a-1,a,a+1]



fig,ax=plt.subplots(2)

Ec=1
delta=1
q0=1.2
T=0.1
deltat=1
N=range(-5,5)
x=np.linspace(-4,4,500)
for i in N:
    ax[0].plot(x,par(i,x,Ec,delta))

n=find_oc(q0,Ec,delta)
ax[0].scatter(q0,par(n,q0,Ec,delta))
ax[0].set_ylim(0,3)
a,b,c=exc(q0,Ec,delta,T,deltat)
z=np.zeros(len(x))
for i in range(len(a)):
    z+=a[i]/((x-b[i])**2+0.1**2)

ax[1].plot(x,z)


#spectra.load(ifar[10])
#xx=np.array(spectra.bias)
#yy=np.array(spectra.conductance)
#ax[1].plot(1000*xx,yy/yy[0])


########


axfreq = fig.add_axes([0.25, 0.0, 0.65, 0.03])
q0s = Slider(
    ax=axfreq,
    label='Max',
    valmin=0,
    valmax=1.5,
    valinit=0,
)

axfreq = fig.add_axes([0.25, 0.03, 0.65, 0.03])
Ts = Slider(
    ax=axfreq,
    label='Max',
    valmin=0,
    valmax=2,
    valinit=0.1,
)


axamp = fig.add_axes([0.03, 0.25, 0.0225, 0.63])
Ecs = Slider(
    ax=axamp,
    label="Min",
    valmin=0,
    valmax=2,
    valinit=1,
    orientation="vertical"
)

axamp = fig.add_axes([0.01, 0.25, 0.0225, 0.63])
etas = Slider(
    ax=axamp,
    label="Min",
    valmin=0,
    valmax=1,
    valinit=0.1,
    orientation="vertical"
)

axamp = fig.add_axes([0.05, 0.25, 0.0225, 0.63])
deltas = Slider(
    ax=axamp,
    label="Min",
    valmin=0,
    valmax=2,
    valinit=1,
    orientation="vertical"
)

def update(val):
    ax[0].clear()
    ax[1].clear()
    for i in N:
        ax[0].plot(x,par(i,x,Ecs.val,deltas.val))

    n=find_oc(q0s.val,Ecs.val,deltas.val)
    ax[0].scatter(q0s.val,par(n,q0s.val,Ecs.val,deltas.val))
    ax[0].set_ylim(0,3)
    a,b,c=exc(q0s.val,Ecs.val,deltas.val,Ts.val,deltat)
    z=np.zeros(len(x))
    for i in range(len(a)):
        z+=a[i]/((x-b[i])**2+etas.val**2)
    ax[1].plot(x,z)
    xx,y=fast_spec(ifar[70])
    ax[1].plot(-x,10*y/y[0])
    print(b)


q0s.on_changed(update)
Ecs.on_changed(update) 
Ts.on_changed(update)  
etas.on_changed(update) 
deltas.on_changed(update)  

#######


0

[3.0, -3.0, 3.0, -3.0, 3.0, -3.0]
[3.9447115384615383, -3.944711538461538, 2.0552884615384617, -3.9447115384615383, 2.0552884615384617, -2.0552884615384617]
[3.9591346153846145, -3.959134615384615, 2.040865384615385, -3.9591346153846145, 2.040865384615385, -2.040865384615385]
[3.951923076923076, -3.9519230769230766, 2.0480769230769234, -3.951923076923076, 2.0480769230769234, -2.0480769230769234]
[1.9759615384615385, -4.024038461538462, 1.9759615384615383, -1.9759615384615385, 5.975961538461538, -1.9759615384615383]
[1.9182692307692306, -4.081730769230769, 1.9182692307692308, -1.9182692307692306, 5.918269230769231, -1.9182692307692308]
[1.903846153846154, -4.096153846153846, 1.9038461538461542, -1.903846153846154, 5.903846153846154, -1.9038461538461542]
[1.875, -4.125, 1.875, -1.875, 5.875, -1.875]
[1.8461538461538458, -4.153846153846154, 1.8461538461538456, -1.8461538461538458, 5.846153846153846, -1.8461538461538456]
[1.8173076923076923, -4.1826923076923075, 1.8173076923076925, -1.8173

# Island By Island

## 1st Island

In [1137]:
h=0
fig, ax = plt.subplots()
Z=[]
for i in file:
    file=glob.glob(path1+'/*SZ01231204*')
    spectra.load(i)
    x=spectra.bias
    y=spectra.conductance
    z=spectra.current
    #plt.plot(x,y/y[0]+0.2*h)
    #plt.scatter(h,z[0])
    h+=1
    Z.append(y/y[0])

val1=1
val2=0
#create the slider
ax.imshow(Z,aspect='auto',vmax=val1,vmin=val2)
fig.subplots_adjust(left=0.25, bottom=0.25)

axfreq = fig.add_axes([0.25, 0.1, 0.65, 0.03])
val1_slider = Slider(
    ax=axfreq,
    label='Max',
    valmin=0.1,
    valmax=10,
    valinit=val1,
)

# Make a vertically oriented slider to control the amplitude
axamp = fig.add_axes([0.1, 0.25, 0.0225, 0.63])
val2_slider = Slider(
    ax=axamp,
    label="Min",
    valmin=-5,
    valmax=4,
    valinit=val2,
    orientation="vertical"
)


val1_slider.on_changed(update)
val2_slider.on_changed(update)  

def update(val):
    ax.clear()
    ax.imshow(Z,aspect='auto',vmax=val1_slider.val,vmin=val2_slider.val)


In [387]:
h=0
fig, ax = plt.subplots()
Z=[]
file=glob.glob(path1+'/*SZ01231204*')
for i in file:
    spectra.load(i)
    x=spectra.bias
    y=spectra.conductance
    z=spectra.current
    y=sc.signal.savgol_filter(y/y[0],7,3)
    ax.plot(x,y+0.2*h)
    h+=1



In [94]:
h=0
#fig, ax = plt.subplots()
#fig2, ax2 = plt.subplots()
Z=[]
Jc=[]

c=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
file=glob.glob(path1+'/*IZ02y120*')
for i in [0,1,2,3,4,5,6,7,8,9]:
    spectra.load(file[i])
    x=np.array(spectra.bias)
    y=np.array(spectra.current)
    x=x*np.abs((y[0]-y[-1])/4e-8)
    z=np.array(spectra.biasVI_b)
    Z.append(spectra.z)
    #z2=spectra.biasVI_f
    n=int(len(x)/2)
    z=z[0:n]
    x=x[0:n]
    temp=-np.gradient(z)
    ax.plot(x,z,color=c[i])
    ax.plot(x,10*temp,color=c[i])
    Jc.append(x[np.where(temp==np.max(temp))[0][0]])
    print(np.abs((y[0]-y[-1])/4e-8))

R=[]
file=glob.glob(path1+'/*SZ01231204*')
for i in [0,1,2,3,4,5,6,7,8,9]:
    
    spectra.load(file[i])
    x=spectra.bias
    y=spectra.conductance
    R.append(x[0]/y[0])

plt.scatter(R,Jc)


0.9610117999999999
0.960170575
0.9546038250000001
0.9470846749999999
0.94146095
0.9350075750000001
0.9258049999999999
0.9169174250000001
0.904395675
0.899770275


In [211]:
fig, ax = plt.subplots()
Z=[]
Jc=[]

file=glob.glob(path1+'/*IZ02y120*')
Y=[]
for i in [0,1,2,3,4,5,6]:
    spectra.load(file[i])
    x=np.array(spectra.bias)
    y=np.array(spectra.current)
    x=x*np.abs((y[0]-y[-1])/4e-8)
    z=np.array(spectra.biasVI_b)
    zz=np.array(spectra.biasVI_f)
    n=int(len(x)/2)
    z1=zz[0:n-1]-zz[n-1]
    x1=x[0:n-1]
    z2=z[n:-1]-z[n]
    x2=x[n:-1]
    ax.plot(x1,(z1+np.flip(z2)),color=c[i])
    #ax.plot(z,z,color=c[i])
    


In [210]:
h=0
fig, ax = plt.subplots()
Z=[]
file=glob.glob(path1+'/*SP01231204*')
for i in [0,1,2,3,4,5,6]:
    spectra.load(file[i])
    x=spectra.bias
    z=spectra.current
    ax.plot(x,z)

## Island 2

In [1239]:
file=glob.glob(path1+'*/SP2-*')
island1=file[0:400]
island2=file[402:547]
island3=file[551:-1]
iclose=[]
ifar=[]
for i in island3:
    spectra.load(i)
    a=spectra.current[0]
    if a>1e-9:
        iclose.append(i)
    else:
        ifar.append(i)
    


In [1240]:
fig, ax=plt.subplots()
Z=[]
for i in range(75,252):
    spectra.load(iclose[i])
    x=np.array(spectra.bias)
    y=np.array(spectra.conductance)
    y=sc.signal.savgol_filter(y/y[0],7,3)
    Z.append(y)

val1=1
val2=0
#create the slider
ax.imshow(Z,aspect='auto',vmax=val1,vmin=val2,extent=[-5,5,0,252])
fig.subplots_adjust(left=0.25, bottom=0.25)

axfreq = fig.add_axes([0.25, 0.1, 0.65, 0.03])
val1_slider = Slider(
    ax=axfreq,
    label='Max',
    valmin=0.1,
    valmax=10,
    valinit=val1,
)

# Make a vertically oriented slider to control the amplitude
axamp = fig.add_axes([0.1, 0.25, 0.0225, 0.63])
val2_slider = Slider(
    ax=axamp,
    label="Min",
    valmin=-5,
    valmax=4,
    valinit=val2,
    orientation="vertical"
)

def update(val):
    ax.clear()
    ax.imshow(Z,aspect='auto',vmax=val1_slider.val,vmin=val2_slider.val,extent=[-5,5,0,252])

val1_slider.on_changed(update)
val2_slider.on_changed(update)  


0

In [1241]:
order=[]

for i in iclose:
    spectra.load(i)
    x=np.array(spectra.bias)
    t=len(x)//2
    x=x[t-50:t+50]
    y=np.array(spectra.conductance)[t-50:t+50]
    y=sc.signal.savgol_filter(y/y[0],9,3)
    #plt.plot(x,y+h)
    a=np.where(y==max(y))
    #plt.scatter(x[a],y[a]+h)
    order.append(x[a])

order, iclose = zip(*sorted(zip(order, iclose)))
order, ifar = zip(*sorted(zip(order, ifar)))

In [1242]:
fig, ax=plt.subplots()


h=0
for i in range(68,252):
    spectra.load(iclose[i])
    x=np.array(spectra.bias)
    y=np.array(spectra.conductance)
    y=sc.signal.savgol_filter(y/np.abs(y[-1]),15,3)
    plt.plot(1000*x,y+0.1*h)
    a=sc.signal.find_peaks(y, height=None, threshold=None, distance=None, prominence=None, width=3, wlen=None, rel_height=0.2, plateau_size=None)[0]
    #a=np.where(y==max(y))
    #ax.scatter(1000*x[a],y[a]/y[a]+h)
    h+=1

#h=0
#for i in range(0,252):
#    spectra.load(iclose[i])
#    x=np.array(spectra.bias)[350:-1]
#    y=np.array(spectra.conductance)[350:-1]
#    y=sc.signal.savgol_filter(y/y[-1],15,3)
#    #plt.plot(1000*x,y+h)
#    a=sc.signal.find_peaks(y, height=None, threshold=None, distance=None, prominence=None, width=3, wlen=None, rel_height=0.2, plateau_size=None)[0]
#    #a=np.where(y==max(y))
#    ax.scatter(1000*x[a],y[a]/y[a]+h)
#    h+=1
#val1=3
#val2=0.1
#create the slider
#ax.imshow(Z,aspect='auto',vmax=val1,vmin=val2,extent=[-5,5,0,252])


## Island 3 (4 islands, comparison)

In [243]:
q0_graphene=[path1+'/S231201_00018.dat',path1+'/S231201_00007.dat',path1+'/S231201_00008.dat',path1+'/S231201_00009.dat',path1+'/S231201_00010.dat',path1+'/S231201_00011.dat']

didvfar1=path1+'/S231130_00047.dat'
didvclose1=path1+'/S231130_00061.dat'
q01=glob.glob(path1+'/*SZ01231130*')[0:21]
I1=glob.glob(path1+'/*I01231130*')[4:9]

didvfar2=path1+'/S231130_00066.dat'
didvclose2=path1+'/S231130_00067.dat'
I2=glob.glob(path1+'/*I231130*')[20:31]

didvfar3=path1+'/S231130_00103.dat'
didvclose3=path1+'/S231130_00022.dat'
I3=glob.glob(path1+'/*Iz01231130*')[3:13]

didvfar4=path1+'/SZ01231291_00024.dat'
q04=[path1+'/S231201_00014.dat',path1+'/S231201_00015.dat',path1+'/S231201_00016.dat',path1+'/S231201_00017.dat']


didvclose5_1=path1+'/S231201_00058.dat'
q05_1=[path1+'/S231201_00059.dat',path1+'/S231201_00060.dat']

didvclose5_2=path1+'/S231201_00061.dat'
q05_2=[path1+'/S231201_00062.dat',path1+'/S231201_00063.dat']

didvclose5_3=path1+'/S231201_00070.dat'
q05_3=[path1+'/S231201_00071.dat',path1+'/S231201_00072.dat']





In [251]:
fig, ax = plt.subplots(2)
x,y=fast_spec(didvclose5_1)
ax[0].plot(x,y/y[-1])
x,y=fast_spec(didvclose5_2)
ax[0].plot(x,y/y[-1])
x,y=fast_spec(didvclose5_3)
ax[0].plot(x,y/y[-1])

for i in range(len(q05_1)):
    x,y=fast_cur(q05_1[i])
    ax[1].plot(x,y,color=color_list[0])
    x,y=fast_cur(q05_2[i])
    ax[1].plot(x,y,color=color_list[1])
    x,y=fast_cur(q05_3[i])
    ax[1].plot(x,y,color=color_list[2])

In [296]:
fig, ax=plt.subplots(ncols=2)
for i in I2:
    spectra.load(i)
    x=spectra.bias
    z=spectra.current
    y=spectra.biasVI_f
    ax[0].plot(x,y)
    ax[1].plot(x,z)

## Isalnd 4

In [371]:
files=np.array(glob.glob(path2+'*/S231207*')[73:90])
files=files[[0,1,2,3,4,5,6,7,8,9,11,12,13,14,15,16]]

h=0
order=[]
for i in files:
    x,y=fast_spec(i)
    y=sc.signal.savgol_filter(y/y[0],7,3)
    #plt.plot(x,y/y[0]+3*h)
    y=np.abs(sc.signal.savgol_filter(np.gradient(y),27,3))
    a=sc.signal.find_peaks(y,height=0.03, threshold=None, distance=None, prominence=None, width=None, wlen=None, rel_height=None, plateau_size=0.5)[0]
    #plt.plot(x,y+h)
    #plt.scatter(x[a],y[a]+h)
    h+=1
    order.append(x[a[-1]])

order, files = zip(*sorted(zip(order, files)))

for i in files:
    x,y=fast_spec(i)
    y=sc.signal.savgol_filter(y/y[0],7,3)
    plt.plot(x,y+h)
    h+=1

In [407]:
files_far=glob.glob(path2+'*/B01*'+'*SP*'+'*01*')
files_close=glob.glob(path2+'*/B01*'+'*SP*'+'*02*')
h=0
for i in files_far:
    x,y=fast_cur(i)
    plt.plot(1000*x,y)
plt.xlim(-0.3,0.3)
plt.hlines(0,-1,1)


## Island 5

In [985]:
files_H0_sp=[glob.glob(path2+'*/SQ08*'+'*H0*'+'*01.*'),glob.glob(path2+'*/SQ08*'+'*H0*'+'*04.*'),glob.glob(path2+'*/SQ08*'+'*H0*'+'*07.*'),glob.glob(path2+'*/SQ08*'+'*H0*'+'*10.*')]
files_H1_sp=[glob.glob(path2+'*/SQ08*'+'*H1*'+'*01.*'),glob.glob(path2+'*/SQ08*'+'*H1*'+'*04.*'),glob.glob(path2+'*/SQ08*'+'*H1*'+'*07.*'),glob.glob(path2+'*/SQ08*'+'*H1*'+'*10.*')]
files_H0_s=[glob.glob(path2+'*/SQ08*'+'*H0*'+'*02.*'),glob.glob(path2+'*/SQ08*'+'*H0*'+'*05.*'),glob.glob(path2+'*/SQ08*'+'*H0*'+'*08.*'),glob.glob(path2+'*/SQ08*'+'*H0*'+'*11.*')]
files_H1_s=[glob.glob(path2+'*/SQ08*'+'*H1*'+'*02.*'),glob.glob(path2+'*/SQ08*'+'*H1*'+'*05.*'),glob.glob(path2+'*/SQ08*'+'*H1*'+'*08.*'),glob.glob(path2+'*/SQ08*'+'*H1*'+'*11.*')]
files_H0_I=[glob.glob(path2+'*/SQ08*'+'*H0*'+'*03.*'),glob.glob(path2+'*/SQ08*'+'*H0*'+'*06.*'),glob.glob(path2+'*/SQ08*'+'*H0*'+'*09.*'),glob.glob(path2+'*/SQ08*'+'*H0*'+'*12.*')]
files_H1_I=[glob.glob(path2+'*/SQ08*'+'*H1*'+'*03.*'),glob.glob(path2+'*/SQ08*'+'*H1*'+'*06.*'),glob.glob(path2+'*/SQ08*'+'*H1*'+'*09.*'),glob.glob(path2+'*/SQ08*'+'*H1*'+'*12.*')]
files_far=glob.glob(path2+'*/S231208*')[120:160]



Phi_0 measurement

In [986]:
PPulse=[]
Cut=[]
Cuter=[]
for j in range(len(files_H0_sp)):
    a=range(len(files_H0_sp[j]))
    Cut1=[]
    Cut2=[]
    Pulse=[]
    for i in a:
        x,y=fast_cur(files_H0_sp[j][i])
        y=sc.signal.savgol_filter(y,27,3)
        Cut1.append(x[find_nearest(y,0.0)])
        Pulse.append(float(files_H0_sp[j][i].split()[-1][13:18]))
        x,y=fast_cur(files_H1_sp[j][i])
        y=sc.signal.savgol_filter(y,27,3)
        Cut2.append(x[find_nearest(y,0.0)])

    Cut1=np.array(Cut1)
    Cut2=np.array(Cut2)

    plt.errorbar(Pulse,(Cut1+Cut2)/2,np.abs((Cut1-Cut2)/2),fmt='o',color=color_list[j])
    PPulse.append(Pulse)
    Cut.append((Cut1+Cut2)/2)
    Cuter.append(np.abs((Cut1-Cut2)/2))



Compare Phi_0 with spectra

In [707]:
fig,ax=plt.subplots(2)
h=0
j=0
for i in a:
    x,y=fast_spec(files_H0_s[j][i])
    y=sc.signal.savgol_filter(y/(y[-1]),27,3)
    ax[0].plot(x,y+0.3*h)
    ax[1].scatter(PPulse[j][i],Cut[j][i])
    h+=1


In [708]:
fig,ax=plt.subplots(1,2)
h=0
j=0
a=range(len(files_H0_sp[j]))
for i in a:
    x,y=fast_spec(files_far[i+len(files_H0_sp[j])*j])
    #y=sc.signal.savgol_filter(y/y[-1],3,3)
    ax[0].plot(1000*x,y/y[-1]+h)
    ax[1].scatter(PPulse[j][i],Cut[j][i])
    h+=1
ax[0].set_xlim(-4,4)

(-4.0, 4.0)

Current bias

In [713]:
fig,ax=plt.subplots(1,2)

j=1
Ip=[]
Im=[]
a=range(len(files_H0_I[j]))
for i in a:
    x,y,z,w=fast_I(files_H1_I[j][i])
    #z=sc.signal.savgol_filter(y/(y[-1]),27,3)
    z=np.gradient(z)
    z=-sc.signal.savgol_filter(z,27,3)
    z=z/max(z)
    maxima1=sc.signal.find_peaks(z, height=0.22, threshold=None, distance=None, prominence=0.11, width=None, wlen=None, rel_height=None, plateau_size=None)[0]
    y=np.gradient(y)
    y=sc.signal.savgol_filter(z,27,3)
    y=y/max(y)
    maxima2=sc.signal.find_peaks(y, height=0.2, threshold=None, distance=None, prominence=0.3, width=None, wlen=None, rel_height=None, plateau_size=None)[0]
    ax[1].plot(x,z)
    ax[1].scatter(x[maxima1],z[maxima1])
    #for h in maxima1:
    #    ax[0].scatter(PPulse[j][i],x[h],color=color_list[i])
    #for h in maxima2:
    #    ax[0].scatter(PPulse[j][i],x[h],color=color_list[i])
    #ax[1].scatter(PPulse[j][i],Cut[j][i])

In [704]:
fig,ax=plt.subplots(1,2)

j=1
Ip=[]
Im=[]
a=range(len(files_H0_I[j]))
for i in a:
    x,y,z,w=fast_I(files_H1_I[j][i])
    #z=sc.signal.savgol_filter(y/(y[-1]),27,3)
    z=np.gradient(z)
    z=-sc.signal.savgol_filter(z,27,3)
    z=z/max(z)
    maxima1=sc.signal.find_peaks(z, height=0.22, threshold=None, distance=None, prominence=0.11, width=None, wlen=None, rel_height=None, plateau_size=None)[0]
    y=np.gradient(y)
    y=sc.signal.savgol_filter(z,27,3)
    y=y/max(y)
    maxima2=sc.signal.find_peaks(y, height=0.2, threshold=None, distance=None, prominence=0.3, width=None, wlen=None, rel_height=None, plateau_size=None)[0]
    #ax[1].plot(x,z)
    #ax[1].scatter(x[maxima1],z[maxima1])
    for h in maxima1:
        ax[0].scatter(PPulse[j][i],x[h],color=color_list[i])
    for h in maxima2:
        ax[0].scatter(PPulse[j][i],x[h],color=color_list[i])
    x,y=fast_spec(files_H0_s[j][i])
    y=sc.signal.savgol_filter(y,7,3)
    ax[1].plot(x,y)

## Island 6

In [987]:
files_H0_sp=[glob.glob(path2+'*/SQ09*'+'*H0*'+'*01.*'),glob.glob(path2+'*/SQ09*'+'*H0*'+'*04.*'),glob.glob(path2+'*/SQ09*'+'*H0*'+'*07.*'),glob.glob(path2+'*/SQ09*'+'*H0*'+'*10.*')]
files_H1_sp=[glob.glob(path2+'*/SQ09*'+'*H1*'+'*01.*'),glob.glob(path2+'*/SQ09*'+'*H1*'+'*04.*'),glob.glob(path2+'*/SQ09*'+'*H1*'+'*07.*'),glob.glob(path2+'*/SQ09*'+'*H1*'+'*10.*')]
files_H0_s=[glob.glob(path2+'*/SQ09*'+'*H0*'+'*02.*'),glob.glob(path2+'*/SQ09*'+'*H0*'+'*05.*'),glob.glob(path2+'*/SQ09*'+'*H0*'+'*08.*'),glob.glob(path2+'*/SQ09*'+'*H0*'+'*11.*')]
files_H1_s=[glob.glob(path2+'*/SQ09*'+'*H1*'+'*02.*'),glob.glob(path2+'*/SQ09*'+'*H1*'+'*05.*'),glob.glob(path2+'*/SQ09*'+'*H1*'+'*08.*'),glob.glob(path2+'*/SQ09*'+'*H1*'+'*11.*')]
files_H0_I=[glob.glob(path2+'*/SQ09*'+'*H0*'+'*03.*'),glob.glob(path2+'*/SQ09*'+'*H0*'+'*06.*'),glob.glob(path2+'*/SQ09*'+'*H0*'+'*09.*'),glob.glob(path2+'*/SQ09*'+'*H0*'+'*12.*')]
files_H1_I=[glob.glob(path2+'*/SQ09*'+'*H1*'+'*03.*'),glob.glob(path2+'*/SQ09*'+'*H1*'+'*06.*'),glob.glob(path2+'*/SQ09*'+'*H1*'+'*09.*'),glob.glob(path2+'*/SQ09*'+'*H1*'+'*12.*')]
files_far=glob.glob(path2+'*/S231208*')[160:180]

In [988]:
PPulse=[]
Cut=[]
Cuter=[]
for j in range(len(files_H0_sp)):   
    a=range(len(files_H0_sp[j]))
    Cut1=[]
    Cut2=[]
    Pulse=[]
    for i in a:
        x,y=fast_cur(files_H0_sp[j][i])
        y=sc.signal.savgol_filter(y,27,3)
        Cut1.append(x[find_nearest(y,0.0)])
        Pulse.append(float(files_H0_sp[j][i].split()[-1][13:18]))
        x,y=fast_cur(files_H1_sp[j][i])
        y=sc.signal.savgol_filter(y,27,3)
        Cut2.append(x[find_nearest(y,0.0)])

    Cut1=np.array(Cut1)
    Cut2=np.array(Cut2)

    plt.errorbar(Pulse,(Cut1+Cut2)*1e6/2,np.abs((Cut1-Cut2)*1e6/2),fmt='o',color=color_list[j])
    PPulse.append(Pulse)
    Cut.append((Cut1+Cut2)/2)
    Cuter.append(np.abs((Cut1-Cut2)/2))
plt.xlabel('Pulse (V)')
plt.ylabel('Voltage cut at I=0 ($\mu$V)')

<>:26: SyntaxWarning: invalid escape sequence '\m'
<>:26: SyntaxWarning: invalid escape sequence '\m'
C:\Users\jortuzar\AppData\Local\Temp\ipykernel_37176\1095545815.py:26: SyntaxWarning: invalid escape sequence '\m'
  plt.ylabel('Voltage cut at I=0 ($\mu$V)')


Text(0, 0.5, 'Voltage cut at I=0 ($\\mu$V)')

In [854]:
fig,ax=plt.subplots(1,2)

m=[0,1,2,3]
for j in m:
    h=0
    a=range(len(files_H0_sp[j]))
    for i in a:
        x,y=fast_spec(files_far[i+len(files_H0_sp[j])*j])
        #y=sc.signal.savgol_filter(y/y[-1],3,3)
        ax[0].plot(1000*x,y/y[-1]+0.2*h+2*j,color=color_list[j])
        ax[1].scatter(PPulse[j][i],Cut[j][i],color=color_list[j])
        h+=1
    ax[0].set_xlim(-4,4)
ax[0].set_xlabel('Bias (meV)')
ax[0].set_ylabel('Conductance (a.u.)')
ax[1].set_xlabel('Pulse (V)')
ax[1].set_ylabel('Voltage cut at I=0 ($\mu$V)')

<>:17: SyntaxWarning: invalid escape sequence '\m'
<>:17: SyntaxWarning: invalid escape sequence '\m'
C:\Users\jortuzar\AppData\Local\Temp\ipykernel_37176\382416464.py:17: SyntaxWarning: invalid escape sequence '\m'
  ax[1].set_ylabel('Voltage cut at I=0 ($\mu$V)')


Text(0, 0.5, 'Voltage cut at I=0 ($\\mu$V)')

In [895]:
ax=[plt.subplot(2,1,1),plt.subplot(2,2,3),plt.subplot(2,2,4)]

j=1
h=0
Ip=[]
Im=[]
a=range(len(files_H0_I[j]))
for i in a:
    x,y,z,w,t=fast_I(files_H1_I[j][i])
    temp=find_nearest(x,0.0)
    z=z-z[temp]
    y=y-y[temp]

    dz=np.gradient(z)
    dz=-sc.signal.savgol_filter(dz,27,3)
    dz=dz/max(dz)
    
    maxima1=sc.signal.find_peaks(dz, height=0.22, threshold=None, distance=None, prominence=0.11, width=None, wlen=None, rel_height=None, plateau_size=None)[0]
    dy=-np.gradient(y)
    dy=sc.signal.savgol_filter(dy,27,3)
    dy=dy/max(dy)
    maxima2=sc.signal.find_peaks(dy, height=0.2, threshold=None, distance=None, prominence=0.11, width=None, wlen=None, rel_height=None, plateau_size=None)[0]
    #ax[0].plot(x,dy)
    #ax[0].scatter(x[maxima2],dy[maxima2])
    for m in maxima1:
        if x[m]>0:
            #ax[0].scatter(PPulse[j][i],x[h],color=color_list[i])
            #ax[0].plot(x,z+1e-3*h,color=color_list[i])
            ax[0].scatter(x[m],z[m],color=color_list[i])
    for m in maxima2:
        if x[m]<0:
            #ax[0].scatter(PPulse[j][i],np.abs(x[h]),color=color_list[i])
            #ax[0].plot(x,y+1e-3*h,color=color_list[i])
            ax[0].scatter(-x[m],-y[m],color=color_list[i])
    x,y=fast_spec(files_H0_s[j][i])

    temp1=find_nearest(x,2e-3)
    temp2=find_nearest(x,-2e-3)
    y=sc.signal.savgol_filter(y/np.abs(y[180]),7,3)[temp1:temp2]
    x=x[temp1:temp2]
    maxima3=sc.signal.find_peaks(y, height=0.2, threshold=None, distance=None, prominence=0.11, width=None, wlen=None, rel_height=None, plateau_size=None)[0]
    ax[1].plot(1000*x,y+0.2*h)
    ax[1].scatter(1000*x[maxima3],y[maxima3]+0.2*h)
    ax[2].scatter(PPulse[j][i],Cut[j][i]*1e6,color=color_list[i])
    h+=1
ax[0].set_xlabel('Current ($\mu$A)')
ax[0].set_ylabel('Bias (meV)')
ax[1].set_xlabel('Bias (meV)')
ax[1].set_ylabel('Conductance (a.u.)')
ax[2].set_xlabel('Pulse (V)')
ax[2].set_ylabel('Voltage cut at I=0 ($\mu$V)')

<>:46: SyntaxWarning: invalid escape sequence '\m'
<>:51: SyntaxWarning: invalid escape sequence '\m'
<>:46: SyntaxWarning: invalid escape sequence '\m'
<>:51: SyntaxWarning: invalid escape sequence '\m'
C:\Users\jortuzar\AppData\Local\Temp\ipykernel_37176\1618880354.py:46: SyntaxWarning: invalid escape sequence '\m'
  ax[0].set_xlabel('Current ($\mu$A)')
C:\Users\jortuzar\AppData\Local\Temp\ipykernel_37176\1618880354.py:51: SyntaxWarning: invalid escape sequence '\m'
  ax[2].set_ylabel('Voltage cut at I=0 ($\mu$V)')


Text(0, 0.5, 'Voltage cut at I=0 ($\\mu$V)')

In [920]:
fig,ax=plt.subplots(1,2)

j=1
h=0
Ip=[]
Im=[]
a=range(len(files_H0_I[j]))
for i in a:
    x,y,z,w,t=fast_I(files_H1_I[j][i])
    temp=find_nearest(x,0.0)
    z=z-z[temp]
    y=y-y[temp]

    dz=np.gradient(z)
    dz=-sc.signal.savgol_filter(dz,27,3)
    dz=dz/max(dz)
    
    maxima1=sc.signal.find_peaks(dz, height=0.22, threshold=None, distance=None, prominence=0.11, width=None, wlen=None, rel_height=None, plateau_size=None)[0]
    dy=-np.gradient(y)
    dy=sc.signal.savgol_filter(dy,27,3)
    dy=dy/max(dy)
    maxima2=sc.signal.find_peaks(dy, height=0.2, threshold=None, distance=None, prominence=0.11, width=None, wlen=None, rel_height=None, plateau_size=None)[0]
    #ax[0].plot(x,dy)
    #ax[0].scatter(x[maxima2],dy[maxima2])
    for m in maxima1:
        if x[m]>0:
            #ax[0].scatter(PPulse[j][i],x[h],color=color_list[i])
            ax[0].plot(x,z+1e-3*h,color=color_list[i])
            ax[0].scatter(x[m],z[m]+1e-3*h,color=color_list[i])
    for m in maxima2:
        if x[m]<0:
            #ax[0].scatter(PPulse[j][i],np.abs(x[h]),color=color_list[i])
            ax[0].plot(x,y+1e-3*h,color=color_list[i])
            ax[0].scatter(x[m],y[m]+1e-3*h,color=color_list[i])
    x,y=fast_spec(files_H0_s[j][i])

    temp1=find_nearest(x,2e-3)
    temp2=find_nearest(x,-2e-3)
    y=sc.signal.savgol_filter(y/np.abs(y[180]),7,3)[temp1:temp2]
    x=x[temp1:temp2]
    maxima3=sc.signal.find_peaks(y, height=0.2, threshold=None, distance=None, prominence=0.11, width=None, wlen=None, rel_height=None, plateau_size=None)[0]
    ax[1].plot(1000*x,y+0.2*h)
    ax[1].scatter(1000*x[maxima3],y[maxima3]+0.2*h)
    h+=1
ax[0].set_xlabel('Current ($\mu$A)')
ax[0].set_ylabel('Bias (meV)')
ax[1].set_xlabel('Bias (meV)')
ax[1].set_ylabel('Conductance (a.u.)')

<>:45: SyntaxWarning: invalid escape sequence '\m'
<>:45: SyntaxWarning: invalid escape sequence '\m'
C:\Users\jortuzar\AppData\Local\Temp\ipykernel_37176\2770295289.py:45: SyntaxWarning: invalid escape sequence '\m'
  ax[0].set_xlabel('Current ($\mu$A)')


Text(0, 0.5, 'Conductance (a.u.)')

## Island 7

In [998]:
files_H0_sp=np.array(glob.glob(path2+'*/B05*'+'*SP*'+'*01.*'))
files_H1_sp=np.array(glob.glob(path2+'*/B05*'+'*SP*'+'*02.*'))
files_H0_I=np.array(glob.glob(path2+'*/B05*'+'*I*'+'*01.*'))
files_H1_I=np.array(glob.glob(path2+'*/B05*'+'*I*'+'*02.*'))
files_H0_s=np.array(glob.glob(path2+'*/B05*'+'*S2*'))
files=[files_H0_sp,files_H1_sp,files_H0_I,files_H1_I,files_H0_s]

In [999]:
for m in range(len(files)):
    Bfield=[]
    for j in range(len(files[m])):
        x=''
        for i in files[m][j][131:-1]:    
            if i=='I':
                break 
            if i=='S':
                break 
            x+=i

        Bfield.append(float(x)) 
    Bfield, files[m] = zip(*sorted(zip(Bfield, files[m])))

In [1017]:
fig, ax= plt.subplots(2)
Cut1=[]
Cut2=[]
for i in range(len(files[0])):   
    x,y=fast_cur(files[0][i])
    y=sc.signal.savgol_filter(y,27,3)
    Cut1.append(x[find_nearest(y,0.0)])
    x,y=fast_cur(files[1][i])
    y=sc.signal.savgol_filter(y,27,3)
    Cut2.append(x[find_nearest(y,0.0)])

Cut1=np.array(Cut1)
Cut2=np.array(Cut2)
Cut=(Cut1+Cut2)/2
Cuter=np.abs((Cut1-Cut2)/2)
Bfield=np.array(Bfield)

ax[0].errorbar(Bfield,Cut*1e6,Cuter*1e6/2,fmt='o')

ax[0].set_xlabel('B field (mT)')
ax[0].set_ylabel('Voltage cut at I=0 ($\mu$V)')

for i in range(len(files[4])):   
    x,y=fast_spec(files[4][i])
    #y=sc.signal.savgol_filter(y,27,3)
    ax[1].plot(x,y/y[0]-1+Bfield[i]/1000)

ax[1].set_ylabel('B field (T)')
ax[1].set_xlabel('Voltage (mV)')

<>:21: SyntaxWarning: invalid escape sequence '\m'
<>:21: SyntaxWarning: invalid escape sequence '\m'
C:\Users\jortuzar\AppData\Local\Temp\ipykernel_37176\2594916960.py:21: SyntaxWarning: invalid escape sequence '\m'
  ax[0].set_ylabel('Voltage cut at I=0 ($\mu$V)')


Text(0.5, 0, 'Voltage (mV)')

In [1033]:
j=1
h=0
Ip=[]
Im=[]
a=range(len(files[2]))
for i in a[1:-1]:
    x,y,z,w,t=fast_I(files[2][i])
    temp=find_nearest(x,0.0)
    z=z-z[temp]
    y=y-y[temp]
    plt.plot(x,z+Bfield[i]*1e-6)
    plt.plot(x,y+Bfield[i]*1e-6)
    
    

## Island 8

In [1035]:
files_H0_sp=np.array(glob.glob(path1+'*/B08*'+'*SP*'+'*01.*'))
files_H1_sp=np.array(glob.glob(path1+'*/B08*'+'*SP*'+'*02.*'))
files_H0_I=np.array(glob.glob(path1+'*/B08*'+'*I*'+'*01.*'))
files_H1_I=np.array(glob.glob(path1+'*/B08*'+'*I*'+'*02.*'))
files_H0_s=np.array(glob.glob(path1+'*/B08*'+'*S2*'))
files=[files_H0_sp,files_H1_sp,files_H0_I,files_H1_I,files_H0_s]

In [1047]:
Bfield=[]
for m in range(len(files)):
    Bfields=[]
    for j in range(len(files[m])):
        x=''
        for i in files[m][j][131:-1]:    
            if i=='I':
                break 
            if i=='S':
                break 
            x+=i

        Bfields.append(float(x)) 
    Bfields, files[m] = zip(*sorted(zip(Bfields, files[m])))
    Bfield.append(Bfields)

In [1052]:
fig, ax= plt.subplots(2)
Cut1=[]
Cut2=[]
for i in range(len(files[0])):   
    x,y=fast_cur(files[0][i])
    y=sc.signal.savgol_filter(y,27,3)
    Cut1.append(x[find_nearest(y,0.0)])
    x,y=fast_cur(files[1][i])
    y=sc.signal.savgol_filter(y,27,3)
    Cut2.append(x[find_nearest(y,0.0)])

Cut1=np.array(Cut1)
Cut2=np.array(Cut2)
Cut=(Cut1+Cut2)/2
Cuter=np.abs((Cut1-Cut2)/2)

ax[0].errorbar(Bfield[0],Cut*1e6,Cuter*1e6/2,fmt='o')

ax[0].set_xlabel('B field (mT)')
ax[0].set_ylabel('Voltage cut at I=0 ($\mu$V)')

for i in range(len(files[4])):   
    x,y=fast_spec(files[4][i])
    #y=sc.signal.savgol_filter(y,27,3)
    ax[1].plot(x,y/y[-1]-1+Bfield[4][i]/1000)

ax[1].set_ylabel('B field (T)')
ax[1].set_xlabel('Voltage (mV)')

<>:20: SyntaxWarning: invalid escape sequence '\m'
<>:20: SyntaxWarning: invalid escape sequence '\m'
C:\Users\jortuzar\AppData\Local\Temp\ipykernel_37176\415506595.py:20: SyntaxWarning: invalid escape sequence '\m'
  ax[0].set_ylabel('Voltage cut at I=0 ($\mu$V)')


Text(0.5, 0, 'Voltage (mV)')

In [ ]:
j=1
h=0
Ip=[]
Im=[]
a=range(len(files[2]))
for i in a[1:-1]:
    x,y,z,w,t=fast_I(files[2][i])
    temp=find_nearest(x,0.0)
    z=z-z[temp]
    y=y-y[temp]
    plt.plot(x,z+Bfield[i]*1e-6)
    plt.plot(x,y+Bfield[i]*1e-6)

## Island 9

In [1236]:
files_H0_sp=np.array(glob.glob(path1+'*/B09*'+'*SP*'+'*01.*'))
files_H1_sp=np.array(glob.glob(path1+'*/B09*'+'*SP*'+'*02.*'))
files_H0_I=np.array(glob.glob(path1+'*/B09*'+'*I*'+'*01.*'))
files_H1_I=np.array(glob.glob(path1+'*/B09*'+'*I*'+'*02.*'))
files_H0_s=np.array(glob.glob(path1+'*/B09*'+'*S2*'))
files=[files_H0_sp,files_H1_sp,files_H0_I,files_H1_I,files_H0_s]

In [1237]:
Bfield=[]
for m in range(len(files)):
    Bfields=[]
    for j in range(len(files[m])):
        x=''
        for i in files[m][j][131:-1]:    
            if i=='I':
                break 
            if i=='S':
                break 
            x+=i

        Bfields.append(float(x)) 
    Bfields, files[m] = zip(*sorted(zip(Bfields, files[m])))
    Bfield.append(Bfields)

In [1238]:
fig, ax= plt.subplots(2)
Cut1=[]
Cut2=[]
for i in range(len(files[0])):   
    x,y=fast_cur(files[0][i])
    y=sc.signal.savgol_filter(y,27,3)
    Cut1.append(x[find_nearest(y,0.0)])
    x,y=fast_cur(files[1][i])
    y=sc.signal.savgol_filter(y,27,3)
    Cut2.append(x[find_nearest(y,0.0)])

Cut1=np.array(Cut1)
Cut2=np.array(Cut2)
Cut=(Cut1+Cut2)/2
Cuter=np.abs((Cut1-Cut2)/2)

ax[0].errorbar(Bfield[0],Cut*1e6,Cuter*1e6/2,fmt='o')

ax[0].set_xlabel('B field (mT)')
ax[0].set_ylabel('Voltage cut at I=0 ($\mu$V)')

for i in range(len(files[4])):   
    x,y=fast_spec(files[4][i])
    #y=sc.signal.savgol_filter(y,27,3)
    ax[1].plot(x,y/y[-1]-1+Bfield[4][i]/1000)

ax[1].set_ylabel('B field (T)')
ax[1].set_xlabel('Voltage (mV)')

<>:20: SyntaxWarning: invalid escape sequence '\m'
<>:20: SyntaxWarning: invalid escape sequence '\m'
C:\Users\jortuzar\AppData\Local\Temp\ipykernel_37176\415506595.py:20: SyntaxWarning: invalid escape sequence '\m'
  ax[0].set_ylabel('Voltage cut at I=0 ($\mu$V)')


Text(0.5, 0, 'Voltage (mV)')

## Island 10

In [1061]:
files_H0_sp=np.array(glob.glob(path1+'*/B10*'+'*SP*'+'*01.*'))
files_H1_sp=np.array(glob.glob(path1+'*/B10*'+'*SP*'+'*02.*'))
files_H0_I=np.array(glob.glob(path1+'*/B10*'+'*I*'+'*01.*'))
files_H1_I=np.array(glob.glob(path1+'*/B10*'+'*I*'+'*02.*'))
files_H0_s=np.array(glob.glob(path1+'*/B10*'+'*S2*'))
files=[files_H0_sp,files_H1_sp,files_H0_I,files_H1_I,files_H0_s]

In [1062]:
Bfield=[]
for m in range(len(files)):
    Bfields=[]
    for j in range(len(files[m])):
        x=''
        for i in files[m][j][131:-1]:    
            if i=='I':
                break 
            if i=='S':
                break 
            x+=i

        Bfields.append(float(x)) 
    Bfields, files[m] = zip(*sorted(zip(Bfields, files[m])))
    Bfield.append(Bfields)

In [1063]:
fig, ax= plt.subplots(2)
Cut1=[]
Cut2=[]
for i in range(len(files[0])):   
    x,y=fast_cur(files[0][i])
    y=sc.signal.savgol_filter(y,27,3)
    Cut1.append(x[find_nearest(y,0.0)])
    x,y=fast_cur(files[1][i])
    y=sc.signal.savgol_filter(y,27,3)
    Cut2.append(x[find_nearest(y,0.0)])

Cut1=np.array(Cut1)
Cut2=np.array(Cut2)
Cut=(Cut1+Cut2)/2
Cuter=np.abs((Cut1-Cut2)/2)

ax[0].errorbar(Bfield[0],Cut*1e6,Cuter*1e6/2,fmt='o')

ax[0].set_xlabel('B field (mT)')
ax[0].set_ylabel('Voltage cut at I=0 ($\mu$V)')

for i in range(len(files[4])):   
    x,y=fast_spec(files[4][i])
    #y=sc.signal.savgol_filter(y,27,3)
    ax[1].plot(x,y/y[-1]-1+Bfield[4][i]/1000)

ax[1].set_ylabel('B field (T)')
ax[1].set_xlabel('Voltage (mV)')

<>:20: SyntaxWarning: invalid escape sequence '\m'
<>:20: SyntaxWarning: invalid escape sequence '\m'
C:\Users\jortuzar\AppData\Local\Temp\ipykernel_37176\415506595.py:20: SyntaxWarning: invalid escape sequence '\m'
  ax[0].set_ylabel('Voltage cut at I=0 ($\mu$V)')


Text(0.5, 0, 'Voltage (mV)')

## Island 11

In [1087]:
files_H0_sp=np.array(glob.glob(path2+'*/B12*'+'*SP*'+'*01.*'))
files_H0_I=np.array(glob.glob(path2+'*/B12*'+'*I*'+'*01.*'))
files_H0_s=np.array(glob.glob(path2+'*/B12*'+'*S2*'))
files=[files_H0_sp,files_H0_I,files_H0_s]

In [1088]:
Bfield=[]
for m in range(len(files)):
    Bfields=[]
    for j in range(len(files[m])):
        x=''
        for i in files[m][j][131:-1]:    
            if i=='I':
                break 
            if i=='S':
                break 
            x+=i
        Bfields.append(float(x)) 
    Bfields, files[m] = zip(*sorted(zip(Bfields, files[m])))
    Bfield.append(Bfields)

In [1093]:
fig, ax= plt.subplots(2)
Cut=[]

for i in range(len(files[0])):   
    x,y=fast_cur(files[0][i])
    y=sc.signal.savgol_filter(y,27,3)
    Cut.append(x[find_nearest(y,0.0)])

Cut=np.array(Cut)

ax[0].scatter(Bfield[0],Cut*1e6)

ax[0].set_xlabel('B field (mT)')
ax[0].set_ylabel('Voltage cut at I=0 ($\mu$V)')

for i in range(len(files[2])):   
    x,y=fast_spec(files[2][i])
    #y=sc.signal.savgol_filter(y,27,3)
    ax[1].plot(x,y/y[-1]-1+Bfield[2][i]/1000)

ax[1].set_ylabel('B field (T)')
ax[1].set_xlabel('Voltage (mV)')

<>:15: SyntaxWarning: invalid escape sequence '\m'
<>:15: SyntaxWarning: invalid escape sequence '\m'
C:\Users\jortuzar\AppData\Local\Temp\ipykernel_37176\808133175.py:15: SyntaxWarning: invalid escape sequence '\m'
  ax[0].set_ylabel('Voltage cut at I=0 ($\mu$V)')


(0.0, 100.0, 200.0, 300.0, 400.0, 500.0, 600.0, 700.0, 800.0, 900.0, 1000.0, 1100.0, 1200.0, 1300.0, 1400.0, 1500.0, 1600.0, 1700.0, 1800.0, 1900.0, 2000.0, 2100.0, 2200.0, 2300.0, 2400.0, 2500.0)


Text(0.5, 0, 'Voltage (mV)')

## Island 12

In [1106]:
files_H0_sp=[glob.glob(path3+'*/SQ01*'+'*H0*'+'*01.*'),glob.glob(path3+'*/SQ01*'+'*H0*'+'*04.*'),glob.glob(path3+'*/SQ01*'+'*H0*'+'*07.*'),glob.glob(path3+'*/SQ01*'+'*H0*'+'*10.*')]
files_H0_s=[glob.glob(path3+'*/SQ01*'+'*H0*'+'*02.*'),glob.glob(path3+'*/SQ01*'+'*H0*'+'*05.*'),glob.glob(path3+'*/SQ01*'+'*H0*'+'*08.*'),glob.glob(path3+'*/SQ01*'+'*H0*'+'*11.*')]
files_H0_I=[glob.glob(path3+'*/SQ01*'+'*H0*'+'*03.*'),glob.glob(path3+'*/SQ01*'+'*H0*'+'*06.*'),glob.glob(path3+'*/SQ01*'+'*H0*'+'*09.*'),glob.glob(path3+'*/SQ01*'+'*H0*'+'*12.*')]
files_far=glob.glob(path2+'*/S231208*')[2:40]


In [1119]:
PPulse=[]
Cut=[]

for j in range(len(files_H0_sp)):
    a=range(len(files_H0_sp[j]))
    cut=[]
    Pulse=[]
    for i in a:
        x,y=fast_cur(files_H0_sp[j][i])
        y=sc.signal.savgol_filter(y,27,3)
        cut.append(x[find_nearest(y,0.0)])
        Pulse.append(float(files_H0_sp[j][i].split()[-1][13:18]))

    cut=np.array(cut)


    plt.scatter(Pulse,cut*1e6,color=color_list[j])
    PPulse.append(Pulse)
    Cut.append(cut)



In [ ]:

fig,ax=plt.subplots(1,2)

m=[0,1,2,3]
for j in m:
    h=0
    a=range(len(files_H0_sp[j]))
    for i in a:
        x,y=fast_spec(files_H0_s[j][i])
        #y=sc.signal.savgol_filter(y/y[-1],3,3)
        ax[0].plot(1000*x,y/y[-1]+0.2*h+2*j,color=color_list[j])
        ax[1].scatter(PPulse[j][i],Cut[j][i]*1e6,color=color_list[j])
        h+=1
    ax[0].set_xlim(-4,4)
ax[0].set_xlabel('Bias (meV)')
ax[0].set_ylabel('Conductance (a.u.)')
ax[1].set_xlabel('Pulse (V)')
ax[1].set_ylabel('Voltage cut at I=0 ($\mu$V)')

## Island 13

In [1371]:
temppath="C:/Users/jortuzar/Desktop/temp"
files_s=glob.glob(temppath+'*/S2*')
files_sp=glob.glob(temppath+'*/SP*')
files_s_new=[]
files_sp_new=[]
for i in files_s:
    files_s_new.append(path2+'/'+i[31:len(i)])
for i in files_sp:
    files_sp_new.append(path2+'/'+i[31:len(i)])


In [1372]:
fig, ax=plt.subplots()

Cut=[]
for i in files_sp_new:
    x,y=fast_cur(i)
    y=sc.signal.savgol_filter(y,27,3)
    Cut.append(x[find_nearest(y,0.0)])
h=0
for i in range(len(files_s)):
    x,y=fast_spec(files_s_new[i])
    ax.plot(x,y/y[0]+h)
    ax.scatter(10*Cut[i],h)
    h+=1
        

# Critical current analysis